### Import all necessary files

In [ ]:
import pyvista as pv

### Define the function for reading any mesh data

In [ ]:
#Check if foam file exists or not
def _get_foam_file(file_path):
    for file in os.listdir(file_path):
        if file.endswith('.foam'):
            return os.path.join(file_path, file)

#Read mesh data using pyvista open foam reader
def _read_mesh_data(work_dir):

    reader = pv.OpenFOAMReader(_get_foam_file(work_dir))
    last_time_step = int(reader.time_values[-1])
    reader.set_active_time_value(last_time_step)
    # reader.cell_to_point_creation = False

    internal_mesh = reader.read()["internalMesh"]
    boundaries = reader.read()["boundary"]

    internal_mesh.work_dir = work_dir
    internal_mesh.active_time_value = last_time_step

    return internal_mesh, boundaries

#Store the selected flow fields in the mesh
def _select_only_relevant_field_data(pvObject, fields):

    pvCopy = pvObject.copy()
    pvCopy.clear_data()
    for field in fields:
        if field in pvObject.array_names:
            pvCopy[field] = pvObject[field]
    
    pvCopy = pvCopy.cell_data_to_point_data() #converting cell data to point data
    return pvCopy

fields = ['epsilon', 'U', 'mag(U)','k', 'strainRate']

### Read the target mesh that is to be interpolated

In [ ]:
target_mesh_dir=r"D:\GPOD ANN Sartorius BioStat 50L 30 degrees removed geoms modified v4\Sartorius BioStat 50L\BIOSTAT_50L_Run_06"
internal_mesh,_=_read_mesh_data(target_mesh_dir)
target_mesh=_select_only_relevant_field_data(internal_mesh, fields)
target_mesh

### Read the largest geometry mesh

In [ ]:
largest_mesh_dir=r"D:\GPOD ANN Sartorius BioStat 50L 30 degrees removed geoms modified v4\Sartorius BioStat 50L\BIOSTAT_50L_Run_41"
largest_mesh,_=_read_mesh_data(largest_mesh_dir)
largest_mesh.clear_data()
largest_mesh

### Interpolate target_mesh against largest_geom with radius=0.001

In [1]:
interp_mesh = largest_mesh.interpolate(target_mesh, radius = 1e-3, progress_bar = True)
interp_mesh = interp_mesh.point_data_to_cell_data()
interp_mesh

NameError: name 'largest_mesh' is not defined

In [ ]:
def visulaizing_patch(true_mesh,interp_mesh,folder_name):
    
    #Slice the true mesh and find the cell centers
    sliced_true_mesh=true_mesh.slice(normal='y')
    sliced_true_mesh_cells=sliced_true_mesh.cell_centers().points
    
    #Slice the interpolated mesh and find the cell ids for sliced true mesh cell centers 
    sliced_interp_mesh=interp_mesh.slice(normal='y')
    sliced_mesh_idx=sliced_interp_mesh.find_containing_cell(sliced_true_mesh_cells)
    
    #Find the cell centers at thos id's
    sliced_interp_mesh_cells=sliced_interp_mesh.cell_centers().points[sliced_mesh_idx]
    
    #Generate a plot to see the where the cell centers are present across the geometry
    pl=pv.Plotter(notebook=True)
    pl.set_background('white')
    #pl.add_mesh(sliced_interp_mesh,opacity=0.6,color='lightblue')
    pl.add_mesh(sliced_interp_mesh,scalars='strainRate', opacity = 0.5,
                 clim=[0.5,100],log_scale=True,show_scalar_bar=True,scalar_bar_args={'color':'black'})
    pl.add_mesh(sliced_true_mesh_cells, color='red', point_size=3, render_points_as_spheres=True)
    pl.add_mesh(sliced_interp_mesh_cells, color='darkblue', point_size=3, render_points_as_spheres=True)
    pl.view_xz()
    
    # Construct the folder path
    global base_folder
    folder_path = os.path.join(base_folder, "Analysis for interpolation verification", folder_name)

    # Create the folder if it doesn't exist
    os.makedirs(folder_path, exist_ok=True)

    pl.screenshot(folder_path + '\\'+ f"interpolation_intersection.png")
        